**Gå tilbake til [← innledningen](miljoanalyseprosjekt.ipynb) eller [del 2 →](del2.ipynb) av prosjektet**

## **Del 1**

I dette prosjektet skal vi gjennomføre oppgaver som fokuserer på oppsett av utviklingsmiljø, innsamling, behandling, analyse, visualisering og prediktiv analyse av miljødata. Oppgave 1 handler om å sette opp et fungerende utviklingsmiljø. Oppgave 2 innebærer å identifisere relevante åpne datakilder og implementere funksjonalitet for å hente data fra disse kildene ved hjelp av Python. I oppgave 3 skal vi utvikle funksjoner for å rense og formatere de innsamlede dataene, med fokus på håndtering av manglende verdier.

Kode nødvendig for å kunne hente funksjoner senere:

In [3]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import pandas as pd

---

## **Oppgave 1: Sett opp utviklingsmiljø**

I VSCode, opprett en ny Jupyter Notebook-fil (med filendelsen «.ipynb») i prosjektmappen. Skriv og kjør kode i den første cellen for å teste at miljøet fungerer som det skal:

Gå til [oppgave 1](oppg1.ipynb) 

---

## **Oppgave 2: Datainnsamling**

### Hvorfor MET?
For dette prosjektet ønsket vi å se på litt ukonvesjonelle data, så vi valgte derfor å se på solskinn varighet. Solskinn kan indikere endringer i skydekke og værmønstre. Det kan også fortelle oss noe om hvor mye solenergi som når bakken som vil påvirke lokal tempratur og som videre kan spille inn på oppvarmingstrender. Spesielt utvikling over tid vil være interessant. 

Før vi bestemte oss for å undersøke sollys var vi først inne å gjode oss kjent med ulike API grensesnitt. Den som sto fram som et godt valg var **The MET weather API**, mer spesefikt valgte vi **Frost API-en** deres. Denne gir gratis tilgang til deres arkiv over historiske vær- og klimadata av stor variasjon og med mye detaljer av kvalitetskontrollerte målinger. 

Les mer her [Hva er Frost?](https://frost.met.no/index.html) og [Om Meteorologisk Institutt](https://www.met.no/en/About-us/About-MET-Norway)

De har en fri og åpen data politikk, med ønske om at dataen de samler kan bli brukt fritt til fordel for samfunnet. Deres kontaktinformasjon, til og med en egen nettside [hvordan komme hit](https://www.met.no/en/contact-us/travel-here) er et godt tegn på legitimitet ettersom de er lett å komme i kontakt med. Det er et statlig etat, med formål om å informere med veldokumentert forsking samt nøytralt og saklig språk. Nettsiden er utformet på en overskitlig og profesjonell måte med mange forklaringer, oppslagsverk og dataklarifikasjoner som gjør forståelse og henting av data letter. 

### Hvorfor CSV-fil?

Frost API bruker JSON-format til data, vi valgte å gjør om og lagre i CSV-fil ettersom det er standard for de fleste analyseverktøy og fungerer for eksempel bra med Pandas, Matplotlib og Plotly. Det er også raskt å laste inn og veldig lesbart. 

Dataen er strukturert slik:

In [4]:
df = pd.read_csv("../data/raa_data_d.csv", nrows=5)
print(df)

                       date  sunshine_hours
0  2016-01-01T00:00:00.000Z             0.9
1  2016-01-02T00:00:00.000Z             1.3
2  2016-01-03T00:00:00.000Z             3.5
3  2016-01-04T00:00:00.000Z             3.6
4  2016-01-05T00:00:00.000Z             2.8


---

## **Oppgave 3: Databehandling**

Her skal vi fokusere på databehandling ved å utvikle funksjoner som renser og formaterer de innsamlede dataene, med særlig vekt på håndtering av manglende verdier og uregelmessigheter ved hjelp av Pandas. I tillegg skal vi benytte teknikker som list comprehensions, iteratorer, pandas og pandas sql (sqldf) for å manipulere dataene effektivt, noe som vil bidra til å forberede dataene for videre analyse.

### Hvilke endringer har vi gjort?

Datasettet vi hentet fra MET var av høy kvalitet uten hull så vi valgte å fjerne og dyplisere data for å igjen kunne gjennopprette disse dataene med estimasjon.

Her er en oversikt over dataen som ble fjernet og duplisert:

In [5]:
from src.finne_mangler import finne_hull

filbane = "../data/raa_data_d.csv"
duplikater, hull = finne_hull(filbane, dato_kolonne="date")


antall totale rader i datasettet: 1819
antall dupliserte datoer: 8
antall datoer som mangler: 16

Dupliserte datoer:
799   2018-03-19 00:00:00+00:00
801   2018-03-20 00:00:00+00:00
803   2018-03-21 00:00:00+00:00
805   2018-03-22 00:00:00+00:00
807   2018-03-23 00:00:00+00:00
809   2018-03-24 00:00:00+00:00
811   2018-03-25 00:00:00+00:00
813   2018-03-26 00:00:00+00:00
Name: date, dtype: datetime64[ns, UTC]

Manglende datoer:
['2016-10-01', '2016-10-02', '2016-10-03', '2017-07-18', '2017-07-19', '2017-07-20', '2017-07-21', '2017-07-22', '2017-07-23', '2017-07-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30']


### Håndtere feil

Vi benytter så **vasket_data** funksjonen for videre kunne fylle disse hullene og fjerne duplikatene, vi valgte lineær interpolasjon som metode her.  

In [2]:
from src.finne_mangler import vasket_data
vasket_data("../data/raa_data_d.csv", dato_kolonne="date", output_fil="utfylt_data.csv")

antall totale rader i datasettet: 1819
antall dupliserte datoer: 8
antall datoer som mangler: 16

Dupliserte datoer:
799   2018-03-19 00:00:00+00:00
801   2018-03-20 00:00:00+00:00
803   2018-03-21 00:00:00+00:00
805   2018-03-22 00:00:00+00:00
807   2018-03-23 00:00:00+00:00
809   2018-03-24 00:00:00+00:00
811   2018-03-25 00:00:00+00:00
813   2018-03-26 00:00:00+00:00
Name: date, dtype: datetime64[ns, UTC]

Manglende datoer:
['2016-10-01', '2016-10-02', '2016-10-03', '2017-07-18', '2017-07-19', '2017-07-20', '2017-07-21', '2017-07-22', '2017-07-23', '2017-07-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30']

Benyttet lineær interpolasjon, fil lagret som : utfylt_data.csv


c:\Users\junoh\Documents\1 DigFor\2 semester\4114 anvendt programering\prog_gr9\proj_environment-main\src\finne_mangler.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_utfylt = pd.concat([df, df_manglende], ignore_index=True)


,date,sunshine_hours
0,2016-01-01 00:00:00+00:00,0.9
1,2016-01-02 00:00:00+00:00,1.3
2,2016-01-03 00:00:00+00:00,3.5
3,2016-01-04 00:00:00+00:00,3.6
4,2016-01-05 00:00:00+00:00,2.8
...,...,...
1822,2020-12-27 00:00:00+00:00,0.0
1823,2020-12-28 00:00:00+00:00,0.0
1824,2020-12-29 00:00:00+00:00,0.6
1825,2020-12-30 00:00:00+00:00,0.0


### Forklaring 

Manglene og uregelmessighetene vi forventet å møte var:
1. manglende datoer
2. duplikerte datoer
3. manglende verider i målingene (NaN)
4. feilformaterte datoer

For å håndtere 1. feil lagde vi funksjonene finne_hull(), mer spesifikt brukte vi pandas pd.date_range() for å finne hvilke datoer det gjelder for å så fyller hullene ved metoden interpolasjon i funksjonen vasket_data().

For å fjerne duplikater datoer brukte vi drop_duplicates().

Manglende målinger blir plukket opp ved hjelp av en løkke hvor alle verdiene får en type middlertidig "missing value" for senere å interpolere dem.

For datoer som kan være i feil format eller tekst brukte vi pd.to_datetime() for å standarisere formatet. 


### Pandas SQL

Med sqldf er det mulig å bruke SQL-spørringer på Pandas DataFrames. Altså kan man gjennomføre komplekse joins, grupperinger og filtreretinger på en veldig lesbar måte. Dette vil være en intuitiv måte å jobbe med dataen på om man er kjent med SQL fra før.

les mer her [using SQL with Pandas DataFrames](https://medium.com/@davidfagb/using-sql-with-pandas-dataframes-1c36f57ea65d)

### List comperhensions
En måte å manipulere data er ved hjelp av list comprehensions, for eksempel konvertere vi DateTimeIndex-en for manglende dato fra finne_hull() som gjorde datoene mer lesbar som 'YYYY-MM-DD'

---



## **Litteraturliste**
*About the Norwegian Meteorological Institute.* (2017, October 2). Www.met.no; Meteorologisk Institutt. https://www.met.no/en/About-us/About-MET-Norway

Bhargawa, A., & Singh, A. K. (2019). Solar irradiance, climatic indicators and climate change – An empirical analysis. *Advances in Space Research*, 64(1), 271–277. https://doi.org/10.1016/j.asr.2019.03.018

Fagbuyiro, D. (2023, October 17). *Using SQL with Pandas DataFrames.* Medium. https://medium.com/@davidfagb/using-sql-with-pandas-dataframes-1c36f57ea65d

Geeksforgeeks. (n.d.-a). *Geeksforgeeks linechart in Matplotlib-Python.* www.Geeksforgeeks.org. https://www.geeksforgeeks.org/line-chart-in-matplotlib-python/

Haigh, J. (2011). *Solar influences on Climate.* Imperial Collage London . https://www.imperial.ac.uk/media/imperial-college/grantham-institute/public/publications/briefing-papers/Solar-Influences-on-Climate---Grantham-BP-5.pdf?utm

Matplotlib. (n.d.-a). *matplotlib Plotting categorical variables.* www.matplotlib.org. https://matplotlib.org/stable/gallery/lines_bars_and_markers/categorical_variables.html#sphx-glr-gallery-lines-bars-and-markers-categorical-variables-py

Matplotlib. (n.d.-a). *Matplotlib constrained layout quide.* www.matplotlib.org. https://matplotlib.org/stable/users/explain/axes/constrainedlayout_guide.html

Python. (n.d.-a). *Python unittest.* www.python.org. https://docs.python.org/3/library/unittest.html

Python. (n.d.-a). *python linechart.* www.python.org https://python-graph-gallery.com/line-chart/

Rouhani, M. (2019). *Applied Programming — Applied Programming.* Ntnu.no. https://rouhani.folk.ntnu.no/textbooks/tdt4114/intro.html

Vilius Dumcius. (2024, November 5). *JSON vs CSV: What’s the Difference?* IPRoyal.com; IPRoyal. https://iproyal.com/blog/json-vs-csv/

W3Schools. (n.d.-a). *Pandas median, gjennomsnitt, standardavvik.* www.w3school.com. https://www.w3schools.com/python/pandas/ref_df_median.asp

W3Schools. (n.d.-a). *Pandas - Removing Duplicates.* Www.w3schools.com. https://www.w3schools.com/python/pandas/pandas_cleaning_duplicates.asp

W3Schools. (n.d.-b). *Pandas DataFrames.* Www.w3schools.com. https://www.w3schools.com/python/pandas/pandas_dataframes.asp

*What is Frost?* (2019). Frost.met.no; Meteorologisk Institutt. https://frost.met.no/index.html

**Gå tilbake til [← innledningen](miljoanalyseprosjekt.ipynb) eller [del 2 →](del2.ipynb) av prosjektet**